## Loading Packages and Tables

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import dask.dataframe as dd
import streamlit as st
import numpy as np

In [2]:
diag = pd.read_csv('C:\\Users\\yahya\\Documents\\GitHub\\Internship\\CSV_files\\diagnoses_table.csv', dtype={"icd_code": "object"})


In [3]:
def detect_icd_version(code):
    code = str(code).strip().upper()
    return 10 if code and code[0].isalpha() else 9

# Apply to your DataFrame
diag['icd_version'] = diag['icd_code'].apply(detect_icd_version)

In [4]:
admission = pd.read_csv('C:\\Users\\Yahya\\Documents\\GitHub\\Internship\\CSV_files\\admissions_table.csv')

In [5]:
diag_adm_table = diag.merge(admission, on=['subject_id', 'hadm_id'])
diag_adm_table = diag_adm_table.sort_values(by="subject_id")
diag_adm_table = diag_adm_table.reset_index(drop=True)

In [6]:
diag_adm_table.shape

(5071195, 6)

## Selecting CML patients

In [7]:
icd9 = diag_adm_table[diag_adm_table['icd_code'].str.match(r'^\d')]
filtered_icd9 = icd9[(icd9['icd_code'].astype(float) >= 20500) & (icd9['icd_code'].astype(float) <= 20592)]

In [8]:
filtered_icd9['icd_code'].value_counts()

20500    924
20501    405
20510    260
20502    176
20511     67
20530     16
20512     10
20581      2
20580      2
20590      1
20531      1
20591      1
Name: icd_code, dtype: int64

In [9]:
filtered_icd9.shape

(1865, 6)

In [10]:
icd10 = diag_adm_table[diag_adm_table['icd_code'].str.match(r'^[A-Z]')]
filtered_icd10 = icd10[icd10['icd_code'].str.startswith('C92')]

In [11]:
filtered_icd10.shape

(2134, 6)

In [12]:
diag_cml = pd.concat([filtered_icd9, filtered_icd10])
diag_cml = diag_cml.reset_index(drop=True)
diag_cml['icd_code'].value_counts()

20500    924
C9200    803
C9201    429
20501    405
C9202    288
20510    260
C9210    219
20502    176
C92Z0     72
C9211     72
20511     67
C9230     55
C92Z1     43
C92Z2     37
C9240     29
20530     16
C9241     15
C92A0     12
C9290     11
C9232     10
C9231     10
20512     10
C9250      9
C9220      6
C9212      4
C9251      2
20580      2
C92A1      2
20581      2
20591      1
20531      1
20590      1
C9261      1
C9260      1
C9242      1
C9221      1
C9252      1
C92A2      1
Name: icd_code, dtype: int64

In [13]:
diag_cml.shape

(3999, 6)

In [14]:
diag_cml

,subject_id,hadm_id,icd_code,icd_version,admittime,dischtime
0,10035631,29654498,20500,9,2113-07-17 17:15:00,2113-07-18 14:55:00
1,10035631,29462354,20500,9,2112-09-17 19:13:00,2112-10-17 01:41:00
2,10035631,20385771,20500,9,2112-12-04 00:00:00,2112-12-27 16:24:00
3,10035631,27496788,20500,9,2113-08-26 17:07:00,2113-08-29 15:18:00
4,10035631,24912093,20501,9,2112-10-22 00:00:00,2112-10-28 12:16:00
...,...,...,...,...,...,...
3994,19880967,24156760,C92A2,10,2175-11-05 19:29:00,2175-11-06 14:00:00
3995,19880967,24192368,C9200,10,2174-04-21 22:32:00,2174-05-22 18:54:00
3996,19912102,28143086,C9210,10,2185-08-16 01:21:00,2185-08-18 15:17:00
3997,19912102,27379197,C9210,10,2183-06-16 22:40:00,2183-07-06 19:35:00


In [15]:
diag_cml.to_csv(r"C:\Users\Yahya\Documents\GitHub\Internship\CSV_files\cml_diagnoses.csv", index=False)

In [68]:
print(f"Unique patient numbers: {diag_cml['subject_id'].nunique()} Leukemia Patients")

Unique patient numbers: 946 Leukemia Patients


## Selecting CVD patients

In [17]:
ICD_9_dict = {
    'hypertensive': range(401, 406),
    'ischemic': range(410, 415),
    'pulmonary': range(415, 418),
    'other': range(420, 430),
    'cerebrovascular': range(430, 439),
    'arterioles': range(440, 450)
}

In [18]:
diag_icd9 = diag_adm_table[diag_adm_table['icd_code'].str.match(r'^\d')]

In [19]:
diag_icd9['icd_code'] = diag_icd9['icd_code'].astype('int64')

c:\Users\Yahya\Documents\GitHub\Internship\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
condition = None

for name, icd_range in ICD_9_dict.items():
    lower = min(icd_range)
    upper = max(icd_range)
    new_condition = (diag_icd9['icd_code'] >= lower) & (diag_icd9['icd_code'] <= upper)
    
    condition = new_condition if condition is None else (condition | new_condition)

diag_cvd_9 = diag_icd9[condition]

In [21]:
def get_icd9_category(icd_code):
    try:
        major = int(str(icd_code)[:3])
        for category, code_range in ICD_9_dict.items():
            if major in code_range:
                return category
    except:
        return None

In [22]:
diag_cvd_9['cvd_category'] = diag_cvd_9['icd_code'].apply(get_icd9_category)

c:\Users\Yahya\Documents\GitHub\Internship\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
diag_cvd_9.shape

(23500, 7)

In [24]:
diag_cvd_9['icd_code'].sort_values().value_counts()

412    15201
414     1950
431     1887
413     1528
417     1231
416      795
430      737
449      105
415       63
436        3
Name: icd_code, dtype: int64

In [25]:
diag_cvd_10 = diag_adm_table[diag_adm_table['icd_code'].str.startswith('I')]

In [26]:
ICD_10_dict = {
    'hypertensive': ['I10', 'I11', 'I12', 'I13', 'I15'],
    'ischemic': ['I20', 'I21', 'I22', 'I23', 'I24', 'I25'],
    'pulmonary': ['I26', 'I27'],
    'other': ['I30', 'I31', 'I33', 'I34', 'I35', 'I36', 'I37', 'I38', 'I39'],
    'cerebrovascular': ['I60', 'I61', 'I62', 'I63', 'I64', 'I65', 'I66', 'I67', 'I68', 'I69'],
    'arterioles': ['I70', 'I71', 'I72', 'I73', 'I74', 'I77']
}

In [27]:
def get_icd10_category(icd_code):
    try:
        for category, prefixes in ICD_10_dict.items():
            if any(icd_code.startswith(p) for p in prefixes):
                return category
    except:
        return None

In [28]:
diag_cvd_10['cvd_category'] = diag_cvd_10['icd_code'].apply(get_icd10_category)


c:\Users\Yahya\Documents\GitHub\Internship\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
diag_cvd = pd.concat([diag_cvd_9, diag_cvd_10])
diag_cvd = diag_cvd.reset_index(drop=True)

In [30]:
diag_cvd.shape

(509603, 7)

In [31]:
diag_cvd

,subject_id,hadm_id,icd_code,icd_version,admittime,dischtime,cvd_category
0,10000764,27897940,412,9,2132-10-14 23:31:00,2132-10-19 16:30:00,ischemic
1,10000980,26913865,412,9,2189-06-27 07:38:00,2189-07-03 03:00:00,ischemic
2,10000980,25242409,412,9,2191-04-03 18:48:00,2191-04-11 16:21:00,ischemic
3,10000980,24947999,412,9,2190-11-06 20:57:00,2190-11-08 15:58:00,ischemic
4,10001176,23334588,412,9,2186-11-29 03:56:00,2186-12-02 15:00:00,ischemic
...,...,...,...,...,...,...,...
509598,19999828,29734428,I10,10,2147-07-18 16:23:00,2147-08-04 18:10:00,hypertensive
509599,19999828,29734428,I8390,10,2147-07-18 16:23:00,2147-08-04 18:10:00,None
509600,19999828,29734428,I9581,10,2147-07-18 16:23:00,2147-08-04 18:10:00,None
509601,19999828,25744818,I10,10,2149-01-08 16:44:00,2149-01-18 17:00:00,hypertensive


In [32]:
diag_cvd['subject_id'].value_counts(dropna=True).unique().sum()

13378

In [33]:
diag_cvd['cvd_category'].value_counts(dropna=False)

NaN                185276
hypertensive       137400
ischemic           101714
cerebrovascular     29951
arterioles          21069
pulmonary           17201
other               16992
Name: cvd_category, dtype: int64

In [34]:
# diag_cvd = diag_cvd.dropna()
diag_cvd = diag_cvd.reset_index(drop=True)
diag_cvd.shape

(509603, 7)

In [35]:
cvd_dummies = pd.get_dummies(diag_cvd['cvd_category'])

In [36]:
diag_cvd = pd.concat([diag_cvd, cvd_dummies], axis=1)
diag_cvd = diag_cvd.drop('cvd_category', axis = 1)

In [37]:
all_patients = pd.DataFrame({'subject_id': diag['subject_id'].unique()})

In [38]:
cvd_dummies['subject_id'] = diag_cvd['subject_id']

In [39]:
cvd_dummies_agg = cvd_dummies.groupby('subject_id').max().reset_index()

In [40]:
diag_cvd = all_patients.merge(cvd_dummies_agg, on='subject_id', how='left')

In [41]:
diag_cvd.fillna(0, inplace=True)

In [42]:
diag_cvd.shape

(141693, 7)

In [43]:
diag_cvd

,subject_id,arterioles,cerebrovascular,hypertensive,ischemic,other,pulmonary
0,10000032,0.0,0.0,0.0,0.0,0.0,0.0
1,10000117,0.0,0.0,0.0,0.0,1.0,0.0
2,10000635,0.0,0.0,1.0,0.0,0.0,0.0
3,10000690,0.0,0.0,0.0,0.0,0.0,0.0
4,10000764,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
141688,19999625,0.0,0.0,0.0,0.0,0.0,0.0
141689,19999784,0.0,0.0,1.0,0.0,0.0,0.0
141690,19999828,0.0,0.0,1.0,0.0,0.0,0.0
141691,19999840,0.0,0.0,0.0,0.0,0.0,0.0


## Merging CML and CVD tables and creating Categories and Dummy Variables

In [44]:
diag_cvd = diag_cvd.merge(admission, on=['subject_id'])
diag_cvd = diag_cvd.reset_index(drop=True)

In [69]:
diag_cml

,subject_id,hadm_id,icd_code,icd_version,admittime,dischtime
0,10035631,29654498,20500,9,2113-07-17 17:15:00,2113-07-18 14:55:00
1,10035631,29462354,20500,9,2112-09-17 19:13:00,2112-10-17 01:41:00
2,10035631,20385771,20500,9,2112-12-04 00:00:00,2112-12-27 16:24:00
3,10035631,27496788,20500,9,2113-08-26 17:07:00,2113-08-29 15:18:00
4,10035631,24912093,20501,9,2112-10-22 00:00:00,2112-10-28 12:16:00
...,...,...,...,...,...,...
3994,19880967,24156760,C92A2,10,2175-11-05 19:29:00,2175-11-06 14:00:00
3995,19880967,24192368,C9200,10,2174-04-21 22:32:00,2174-05-22 18:54:00
3996,19912102,28143086,C9210,10,2185-08-16 01:21:00,2185-08-18 15:17:00
3997,19912102,27379197,C9210,10,2183-06-16 22:40:00,2183-07-06 19:35:00


In [70]:
diag_cvd_unique = diag_cvd[['subject_id', 'arterioles', 'cerebrovascular', 'hypertensive', 'ischemic', 'other', 'pulmonary']].drop_duplicates()
diag_cvd_unique = diag_cvd_unique.reset_index(drop=True)
diag_cvd_unique

,subject_id,arterioles,cerebrovascular,hypertensive,ischemic,other,pulmonary
0,10000032,0.0,0.0,0.0,0.0,0.0,0.0
1,10000117,0.0,0.0,0.0,0.0,1.0,0.0
2,10000635,0.0,0.0,1.0,0.0,0.0,0.0
3,10000690,0.0,0.0,0.0,0.0,0.0,0.0
4,10000764,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
141688,19999625,0.0,0.0,0.0,0.0,0.0,0.0
141689,19999784,0.0,0.0,1.0,0.0,0.0,0.0
141690,19999828,0.0,0.0,1.0,0.0,0.0,0.0
141691,19999840,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
diag_cvd

,subject_id,arterioles,cerebrovascular,hypertensive,ischemic,other,pulmonary,hadm_id,admittime,dischtime
0,10000032,0.0,0.0,0.0,0.0,0.0,0.0,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00
1,10000117,0.0,0.0,0.0,0.0,1.0,0.0,22927623,2181-11-15 02:05:00,2181-11-15 14:52:00
2,10000117,0.0,0.0,0.0,0.0,1.0,0.0,27988844,2183-09-18 18:10:00,2183-09-21 16:30:00
3,10000635,0.0,0.0,1.0,0.0,0.0,0.0,20642640,2143-12-23 14:55:00,2143-12-24 12:52:00
4,10000635,0.0,0.0,1.0,0.0,0.0,0.0,26134563,2136-06-19 14:24:00,2136-06-20 11:30:00
...,...,...,...,...,...,...,...,...,...,...
354563,19999828,0.0,0.0,1.0,0.0,0.0,0.0,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00
354564,19999828,0.0,0.0,1.0,0.0,0.0,0.0,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00
354565,19999840,0.0,0.0,0.0,0.0,0.0,0.0,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00
354566,19999840,0.0,0.0,0.0,0.0,0.0,0.0,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00


In [72]:
# Keep only columns we need
cml = diag_cml[['subject_id', 'admittime']].rename(columns={'admittime': 'cml_date'})
cvd = diag_cvd[['subject_id', 'admittime']].rename(columns={'admittime': 'cvd_date'})


In [82]:
cml['cml_date'] = pd.to_datetime(cml['cml_date'])
cvd['cvd_date'] = pd.to_datetime(cvd['cvd_date'])

In [85]:
cml.head(25)

,subject_id,cml_date
0,10035631,2113-07-17 17:15:00
1,10035631,2112-09-17 19:13:00
2,10035631,2112-12-04 00:00:00
3,10035631,2113-08-26 17:07:00
4,10035631,2112-10-22 00:00:00
5,10035631,2112-11-10 15:55:00
6,10070928,2187-07-17 12:03:00
7,10070928,2188-04-08 05:21:00
8,10070928,2187-06-07 16:08:00
9,10070928,2187-03-09 21:00:00


In [75]:
cml['subject_id'].nunique()

946

In [76]:
cvd['subject_id'].nunique()

141693

In [77]:
cml_cvd = cml.merge(cvd, on='subject_id', how='left')
cml_cvd = cml_cvd.reset_index(drop=True)

In [78]:
cml_cvd

,subject_id,cml_date,cvd_date
0,10035631,2113-07-17 17:15:00,2112-12-04 00:00:00
1,10035631,2113-07-17 17:15:00,2115-11-08 13:54:00
2,10035631,2113-07-17 17:15:00,2116-02-13 11:17:00
3,10035631,2113-07-17 17:15:00,2112-11-10 15:55:00
4,10035631,2113-07-17 17:15:00,2112-10-22 00:00:00
...,...,...,...
48061,19912102,2183-06-16 22:40:00,2185-08-16 01:21:00
48062,19922600,2131-01-04 19:55:00,2127-12-13 12:03:00
48063,19922600,2131-01-04 19:55:00,2127-09-20 13:46:00
48064,19922600,2131-01-04 19:55:00,2130-12-18 01:08:00


In [79]:
### check if all CML patients are in the merged table
cml[cml['subject_id'].isin(cml_cvd['subject_id'].unique())]['subject_id'].nunique()

946

In [80]:
cml_cvd['has_future_cvd'] = (cml_cvd['cvd_date'] > cml_cvd['cml_date']).astype(int)
cml_cvd['has_immediate_cvd'] = (cml_cvd['cvd_date'] == cml_cvd['cml_date']).astype(int)
cml_cvd['has_cvd'] = (cml_cvd['cvd_date'] >= cml_cvd['cml_date']).astype(int)
cml_cvd['has_cvd_before_cml'] = (cml_cvd['cvd_date'] < cml_cvd['cml_date']).astype(int)
### Calculate the durations (by months) between CVD diagnosis and CML diagnosis
cml_cvd['duration_cvd'] = (cml_cvd['cvd_date']-cml_cvd['cml_date']).dt.days / 30.44

In [81]:
labels_has_future_cvd = cml_cvd.groupby('subject_id')['has_future_cvd'].max().reset_index()
labels_has_immediate_cvd = cml_cvd.groupby('subject_id')['has_immediate_cvd'].max().reset_index()
labels_has_cvd = cml_cvd.groupby('subject_id')['has_cvd'].max().reset_index()
labels_has_cvd_before_cml = cml_cvd.groupby('subject_id')['has_cvd_before_cml'].max().reset_index()
cml_cvd_duration = cml_cvd.groupby('subject_id')['duration_cvd'].min().reset_index()

print(f"Patients got CVD after CML: \n{labels_has_future_cvd.has_future_cvd.value_counts()}")
print(f"Patients got CVD with CML: \n{labels_has_immediate_cvd.has_immediate_cvd.value_counts()}")
print(f"Patients got CVD before CML: \n{labels_has_cvd_before_cml.has_cvd_before_cml.value_counts()}")
print(f"Patients got CVD after or with CML: \n{labels_has_cvd.has_cvd.value_counts()}")
print(f"Patients NO CVD with CML: \n{cml_cvd_duration['duration_cvd'].isnull().sum()}")

Patients got CVD after CML: 
1    593
0    353
Name: has_future_cvd, dtype: int64
Patients got CVD with CML: 
1    946
Name: has_immediate_cvd, dtype: int64
Patients got CVD before CML: 
1    659
0    287
Name: has_cvd_before_cml, dtype: int64
Patients got CVD after or with CML: 
1    946
Name: has_cvd, dtype: int64
Patients NO CVD with CML: 
0


In [ ]:
diag_cml_labeled = diag_cml.merge(labels_has_cvd, on='subject_id', how='left')
diag_cml_labeled = diag_cml_labeled.merge(cml_cvd_duration, on='subject_id', how='left')

In [ ]:
diag_cml_labeled_unique = diag_cml_labeled[['subject_id', 'has_cvd', 'duration_cvd']].drop_duplicates()
diag_cml_labeled_unique['has_cvd'].value_counts()
# diag_cml_labeled['has_future_cvd'] = diag_cml_labeled['has_future_cvd'].fillna(0).astype(int)

In [ ]:
diag_cml_labeled_unique = diag_cml_labeled_unique.merge(diag_cvd_unique, on='subject_id')
diag_cml_labeled_unique = diag_cml_labeled_unique.reset_index(drop=True)
diag_cml_labeled_unique

In [ ]:
cml[cml['subject_id'].isin(diag_cvd_unique['subject_id'].unique())]['subject_id'].nunique()

In [ ]:
diag_cml_labeled_unique['has_cvd'].value_counts()

In [ ]:
from pywaffle import Waffle

values = diag_cml_labeled_unique['has_cvd'].value_counts().to_dict()

fig = plt.figure(
    FigureClass=Waffle,
    rows=5,
    values=values,
    colors=["#d8562f", "#8cd82f"],
    block_arranging_style='snake',
    figsize=(15, 10),
    legend={
        'loc': 'upper left',
        'bbox_to_anchor': (1, 1),
        'fontsize': 12,
        'title': 'CVD Status'
    }
)


## Loading and Preprosessing Labevents data

In [ ]:
lab = pd.read_csv(r'C:\Users\Yahya\Documents\GitHub\Internship\CSV_files\labevents_combined.csv')

In [ ]:
diag_cml_lab = diag_cml_labeled_unique.merge(lab, on=['subject_id'])

In [ ]:
diag_cml_lab = diag_cml_lab.reset_index(drop=True)

In [ ]:
diag_cml_lab = diag_cml_lab.drop(columns=['hadm_id', 'charttime'])

In [ ]:
diag_cml_lab = diag_cml_lab.drop_duplicates()

In [ ]:
diag_cml_lab

## Loading and Preprosessing Pharmacy data

In [ ]:
pharm = pd.read_csv(r'C:\Users\Yahya\Documents\GitHub\Internship\CSV_files\pharmacy_table.csv')

In [ ]:
pharm.rename(columns={'duration':'duration_med'}, inplace=True)

In [ ]:
pharm = pharm.drop(columns=['starttime', 'stoptime', 'hadm_id'])

In [ ]:
drug_mapping = {
    'imatinib': 'Imatinib',
    'imatinib mesylate': 'Imatinib',
    '*nf* imatinib mesylate': 'Imatinib',
    'imatinib oral suspension': 'Imatinib',
    'imatinib (gleevec)': 'Imatinib',
    'dasatinib': 'Dasatinib',
    'dasatinib 20mg tablet': 'Dasatinib',
    '*nf* dasatinib': 'Dasatinib',
    'ruxolitinib': 'Ruxolitinib',
    'inv-ruxolitinib': 'Ruxolitinib',
    'nilotinib': 'Nilotinib',
    '*nf* nilotinib': 'Nilotinib',
    'cabozantinib': 'Cabozantinib',
    'inv-cabozatinib': 'Cabozantinib',
    'ruxolitinib ':'Ruxolitinib',
    'inv-cabozantinib (cabozantinib)': 'Cabozantinib',
    'crizotinib': 'Crizotinib'
}

In [ ]:
pharm['medication_norm'] = pharm['medication'].str.lower().map(drug_mapping)


In [ ]:
pharm['medication_norm'].value_counts(dropna=False)

In [ ]:
doses = pd.read_csv(r'C:\Users\Yahya\Documents\GitHub\Internship\CSV_files\drug_doses.csv')
doses

In [ ]:
doses.rename(columns={'dose_val_rx': 'prescribed_dose'}, inplace=True)

In [ ]:
doses['medication_norm'] = doses['drug'].str.lower().map(drug_mapping)

In [ ]:
doses

In [ ]:
pharm = pharm.merge(
    doses[['subject_id', 'medication_norm', 'prescribed_dose']],
    how='left',
    on=['subject_id', 'medication_norm']
)

In [ ]:
pharm

In [ ]:
pharm_dummies = pd.get_dummies(pharm['medication_norm'])

In [ ]:
pharm = pd.concat([pharm, pharm_dummies], axis=1)

In [ ]:
pharm = pharm.drop(columns=['medication', 'medication_norm'])
pharm = pharm.reset_index(drop=True)

In [ ]:
pharm[['Cabozantinib', 'Crizotinib', 'Dasatinib', 'Imatinib', 'Nilotinib', 'Ruxolitinib']].value_counts()

In [ ]:
pharm_diag = diag_cml_labeled_unique.merge(pharm, on=['subject_id'])

In [ ]:
pharm = pharm_diag.drop_duplicates()

In [ ]:
pharm

In [ ]:
import gc

gc.collect()

In [ ]:
diag_cml_lab.to_csv(r'C:\Users\Yahya\Documents\GitHub\Internship\CSV_files\diag_cml_lab')

In [ ]:
pharm.to_csv(r'C:\Users\Yahya\Documents\GitHub\Internship\CSV_files\pharm')